In [ ]:
import re
import pickle
import pandas as pd

with open('list', 'rb') as f:
	l = pickle.load(f)

In [ ]:
import requests
from tqdm import tqdm

s = requests.Session()
for p in tqdm(l):
	with open(p, 'w') as f:
		f.write(s.get("https://www.felixcloutier.com/x86/" + p).text)

In [ ]:
for p in tqdm(l):
	with open(p, 'r+') as f:
		tmp = f.read()
		tmp = tmp[tmp.find('<table'):tmp.find('</table>') + 8]
		f.seek(0)
		f.write(tmp)
		f.truncate()

In [ ]:
things = []
for file in l:
	with open(file, 'r') as f:
		df = pd.read_html(f.read())[0]
	# if 'Opcode/Instruction' in df.keys():
	# 	things.append(df)
	if 'Opcode**' in df.keys() and 'Instruction' in df.keys():
		thing = []
		for i in range(df.shape[0]):
			thing.append(str(df['Opcode**'][i]) + ' ' + str(df['Instruction'][i]))
		df.insert(0, 'Opcode/Instruction', thing, True)
		df.drop(['Opcode**', 'Instruction'], 1, inplace=True)
		things.append(df)
	if 'Opcode***' in df.keys() and 'Instruction' in df.keys():
		thing = []
		for i in range(df.shape[0]):
			thing.append(str(df['Opcode***'][i]) + ' ' + str(df['Instruction'][i]))
		df.insert(0, 'Opcode/Instruction', thing, True)
		df.drop(['Opcode***', 'Instruction'], 1, inplace=True)
		things.append(df)

In [ ]:
for thing in things:
	for i in range(thing.shape[0] - 1, -1, -1):
		if thing['Opcode/Instruction'][i].startswith('VEX') or thing['Opcode/Instruction'][i].startswith('EVEX'):
			thing.drop(i, inplace=True)
			continue
		# remove 64-bit incompatible instructions (in column 2)
		if not thing.iloc[i][2].startswith('V'):
			thing.drop(i, inplace=True)
things

In [ ]:
for i in range(len(things) - 1, -1, -1):
	if (things[i].shape[0] == 0):
		things.pop(i)
things

In [ ]:
for thing in things:
	for i in range(thing.shape[1] - 1, 0, -1):
		thing.drop(thing.columns[-1], axis=1, inplace=True)
things[1]

In [ ]:
things[24]

In [ ]:
s = []
for thing in things:
	for entry in thing['Opcode/Instruction']:
		s.append(entry)
s

In [ ]:
tmp = [str(line) for line in s]
for i, line in enumerate(s):
	a = line.split('                                                                                     ')
	a[1] = a[1].replace(' ', '')
	s[i] = a[0] + ' ' + a[1]
s

In [ ]:
# s = tmp
[print(x) for x in s]

In [ ]:
new = [x for x in s]
s.sort()
prev = s[0].split(' ')[0]
for line in s:
	if line.split(' ')[0] != prev:
		print()
	prev = line.split(' ')[0]
	print(line)
s = new